In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME - completed #####
# changed animal_shelter and AnimalShelter to match CRUD Python module file name and class name
from crud_animal import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIXME - completed - updated with username and password and CRUD Python module name

username = "aacUser"
password = "Andy12"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Module Six Milestone')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Andreas Galatis CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className = 'row',
        style = {'display' : 'flex'},
            children = [
                html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Cats'),
                html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Dogs')
            ]),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME - completed: Set up the features for interactive data table to make it user-friendly for client
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    html.H2('Andreas Galatis CS-340 Dashboard Visualizations'),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(Output('datatable-id', "data"),
             [Input('submit-button-one', 'n_clicks'), 
              Input('submit-button-two', 'n_clicks'),
             ])

# Added interactive filter options (two call buttons to filter between dogs & cats)
def on_click(bt1, bt2):
    # start case
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.read())
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Cat"})))
    elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Dog"})))
        
    return df.to_dict('records')
                                             


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
#FIXME-completed - Added in the code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Autstin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width': '1000px', 'height': '500px'}, 
               center = [30.75, -97.48],
               zoom = 10,
               children = [dl.TileLayer(id = "base-layer-id"), 
                # Marker with tool tip and popup
                        dl.Marker(position = [30.75, -97.48],
                        children = [dl.Tooltip(dff.iloc[0,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[1,9])
                        ])
                ])
         ])
    ]

app